Importing packages Pandas (python library for data structutres and analytics) and two packages to call Google Analytics API.

In [ ]:
import pandas as pd
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

Calling the key(password that authorizes access to the data) and the right view(to access report and analysis tools).

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '        '
VIEW_ID = '        '

Google API's documentation for web traffic for product's each page.

In [ ]:

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics

In [ ]:

def get_report(analytics, pageTokenVar):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '600daysAgo', 'endDate': 'yesterday'}],
          'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:Pageviews'}], #, {'expression': 'ga:Goal completions'} didn't work.
          'dimensions': [{'name': 'ga:Date'}],
          'pageSize': 10000,
          'pageToken': pageTokenVar,
          'samplingLevel': 'LARGE'
        }]
      }
  ).execute()

Code for PageViews to perform analysis later.

In [ ]:
def handle_report(analytics,pagetoken,rows):  
    response = get_report(analytics, pagetoken) 
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:

        nicerows=[]
        for row in rows:
            dic={}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dic[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dic[metric.get('name')] = float(value)
                    else:
                        dic[metric.get('name')] = int(value)
            nicerows.append(dic)
        return nicerows

Expressing PageViews in DataFrame for analysis

In [ ]:

def main():    
    analytics = initialize_analyticsreporting()
     
    global dfanalytics
    dfanalytics = []
 
    rows = []
    rows = handle_report(analytics,'0',rows)
 
    dfanalytics = pd.DataFrame(list(rows))
 
 
main()
 
print(dfanalytics)

Predicting web traffic for future analysis.

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot

Changing the value column to 'y' and timestanp to 'ds' to make it a DateTime variable

In [ ]:
df = dfanalytics
df['ds'] = pd.to_datetime(df['ga:Date'])
df.rename(columns={'ga:sessions': 'y'}, inplace= True)
df.drop(['ga:Date', 'ga:Pageviews'], axis=1, inplace=True)

Creating a web traffic model and training it on the previous data obtained

In [ ]:
model = Prophet(changepoint_prior_scale=0.5, changepoint_range=0.9, seasonality_mode='multiplicative', yearly_seasonality = 10)
model.fit(df)

Creating future timestamps to predict future web traffic

In [ ]:
future = model.make_future_dataframe(periods= 60)
future['cap'] = 120
future['floor'] = 0
print(future.tail())

Forecasting the website traffic

In [ ]:
forecast = model.predict(future)
model.plot(forecast)

Also
we can see overall trends compare to data and see where the data changes

In [ ]:
fig = model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), model, forecast)

We can also see different components in our plot like weekly and yearly trends and make conclusions from it

In [ ]:
model.plot_components(forecast)

We can also visualize our data through different plots

Importing Data Visualization packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from matplotlib import style
from datetime import datetime
from matplotlib.animation import FuncAnimation, PillowWriter
%matplotlib inline

Plot based on our data and forecast

In [ ]:
with plt.style.context('dark_background'):
    fig = plt.figure(figsize=(10,10))
    plt.subplot(1,1,1)
    plt.plot(forecast['ds'], forecast['yhat'], linewidth=0,marker="o")